#### Hello World Graph

##### Graph One

In [2]:
from typing import Dict, TypedDict
from langgraph.graph import StateGraph

In [3]:
#State
class AgentState(TypedDict):
    message : str

#Node
def greeting_node(state:AgentState)->AgentState:
    #dock string - tells the LLM what this node is doing
    """Simple node that adds a greeting message to the state"""
    state['message'] = f"Hey {state['message']}, how are you doing!" 
    return state

In [6]:
#Define Graph
graph = StateGraph(state_schema=AgentState)
#add node
graph.add_node("greeter", action=greeting_node)
#start pioint
graph.set_entry_point("greeter")
#end point
graph.set_finish_point("greeter")

app = graph.compile()

In [7]:
type(app), type(graph)

(langgraph.graph.state.CompiledStateGraph, langgraph.graph.state.StateGraph)

In [8]:
app.invoke({'message':'Bob'})

{'message': 'Hey Bob, how are you doing!'}

#### Graph One Exercise

In [10]:
class AgentState(TypedDict):
    name : str

def node_one(state:AgentState)->AgentState:
    """This is node one"""
    name = f"{state['name']}, you're doing an amazing job learning LangGraph!"
    return AgentState(name=name)

In [11]:
graph = StateGraph(state_schema=AgentState)
graph.add_node(node='node one', action=node_one)
graph.set_entry_point('node one')
graph.set_finish_point('node one')

app = graph.compile()

In [12]:
app.invoke({'name':'Bob'})

{'name': "Bob, you're doing an amazing job learning LangGraph!"}